<br>

# Introdução

Inicialmente faz-se necessário importar os pacotes utilizados, bem como definir as funções que serão usadas posterirmente.

*Script* criado para organizar uma tabela com a estimativa da população de cada município do Brasil. Os dados foram obtidos com auxílio da [plataforma SIDRA](https://sidra.ibge.gov.br/pesquisa/EstimaPop/tabelas), obtendo os dados da [tabela 6579](https://sidra.ibge.gov.br/tabela/6579) e ajustando-os, elimintando cabeçalhos e rodapés etc.

In [4]:
import os
import requests
import traquitanas.utils as tt
import pandas as pd

In [ ]:
input_path = os.path.join('..', 'data')
os.makedirs(input_path, exist_ok=True)

<br>

## *Download* dos dados

Deve-se definir a *url* que se localizam os dados, bem como o nome do arquivo a ser baixado. A *url* foi definida por meio do [modo de consulta do SIDRA](https://sidra.ibge.gov.br/tabela/6579), onde se definiu o *layout* da tabela e posterirmente acessando a função *Links de Compartilhar*.

O *layout* foi definido selecionando-se todos os anos disponíveis e os níveis territoriais foi selecionado apenas o nivel mais desagregado, que possibilita agregar os dados conforme o usuário quiser, ou seja, municípios.

Inicialmente está se trabalhando com o *download* de arquivos **_.csv_**, porém pretende-se futuramente tentar acessar os dados pela API do SIDRA (https://apisidra.ibge.gov.br/values/t/6579/n6/all/v/all/p/all)

![optionSIDRA](https://i.imgur.com/eCVDpEm.png) <!-- .element height="20%" width="20%" style="border: 0; background: None; box-shadow: None" -->

In [ ]:
url = 'https://sidra.ibge.gov.br/geratabela'

parameters = {
    'format': 'br.csv',
    'name': 'tabela6579.csv',
    'terr': 'NC',
    'rank': '-',
    'query': 't/6579/n6/all/v/all/p/all/l/v,p,t',
}

r = requests.get(url, params=parameters)
print(r.url)

In [ ]:
filename = os.path.join(input_path, 'tab_estimativapopulação.csv')

with open(filename, 'wb') as f:
    f.write(r.content)

<br>

## Lê e Ajusta a Tabela

Para edições e limpeza da tabela, os dados são lidos, aonde já se define parâmetros para pular linhas iniciais, ignorar as notas de rodapé etc.

In [ ]:
file_encoding = tt.predict_encoding(filename)
print(file_encoding)

df = pd.read_csv(
    filename,
    delimiter=';',
    header=1,
    skiprows=range(1, 3),
    nrows=5570,
    #encoding=file_encoding
)

In [ ]:
# Ajusta o Índice
df.rename(
    {
        'Cód.': 'ID_Município',
        'Município': 'Município_Orig'},
    axis=1,
    inplace=True
)

# Set Index
df = df.set_index('ID_Município')

# Cria/desmembra colunas
df['Município'] = df['Município_Orig'].str.split(' \(').str[0]
df['Estado'] = df['Município_Orig'].str.split(' \(').str[1]
df['Estado'] = df['Estado'].replace('\)', '', regex=True)

# Exclui coluna
df.drop(['Município_Orig'], axis=1, inplace=True)

# Ajusta a ordem das colunas
col = df.columns.to_list()
col.insert(0, col.pop(col.index('Município')))
col.insert(1, col.pop(col.index('Estado')))
df = df.reindex(columns=col)

# Results
df.head()

In [ ]:
# Lista de Colunas a serem alteradas
list_columns = df.columns[2:]
list_columns

In [ ]:
# Função que Altera
for i in list_columns:
    year = str(i)
    df[year] = pd.to_numeric(df[year], errors='coerce', downcast='float')

In [ ]:
# Resultados
print(df.dtypes)
df.head()

<br>

## Escreve a Tabela

In [ ]:
df.to_csv(
    os.path.join(input_path, 'tab_EstimativaPopulação_Ajusted.csv'),
    encoding='utf-8-sig',
    index=True,
    header=True,
    #sep=';',
    #decimal=','
)    